In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ydata_profiling as pp
import seaborn as sns
import warnings
import os
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.ensemble import AdaBoostRegressor, VotingRegressor, GradientBoostingRegressor, StackingRegressor, RandomForestRegressor
from sklearn.feature_selection import SelectPercentile, mutual_info_regression, VarianceThreshold, SelectFromModel
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import Ridge, LinearRegression,LogisticRegression
from sklearn.metrics import make_scorer, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import RobustScaler

data = pd.read_csv('../triumphventure/data/clean_data.csv', encoding= 'unicode_escape')

In [2]:
# Drop the unnamed column
data = data.drop(columns=['Unnamed: 0'])

In [3]:
#X_strong_features_test = X_test_new.drop(columns=list(weak_features))
y_new = data["status"].astype(int)
X_new = data.drop(columns=["status"])
from sklearn.model_selection import train_test_split
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)



In [4]:
class columnDropperTransformer(TransformerMixin, BaseEstimator):
    def __init__(self):
        self.weak_features = ['Industry_Group_Gaming', 'Industry_Group_Professional Services',
       'Industry_Group_Financial Services', 'Industry_Group_Advertising',
       'Industry_Group_Mobile', 'Industry_Group_Media and Entertainment',
       'Industry_Group_Privacy and Security', 'Industry_Group_Education',
       'Industry_Group_Content and Publishing',
       'Industry_Group_Sales and Marketing',
       'Industry_Group_Manufacturing',
       'Industry_Group_Consumer Electronics',
       'Industry_Group_Data and Analytics',
       'Industry_Group_Transportation', 'Industry_Group_Hardware',
       'Industry_Group_Clothing and Apparel',
       'Industry_Group_Real Estate', 'Industry_Group_Design',
       'Industry_Group_Food and Beverage',
       'Industry_Group_Travel and Tourism', 'Industry_Group_Platforms',
       'Industry_Group_Sports', 'Industry_Group_Events',
       'Industry_Group_Consumer Goods',
       'Industry_Group_Administrative Services', 'Industry_Group_Energy',
       'Industry_Group_Sustainability',
       'Industry_Group_Messaging and Telecommunication',
       'Industry_Group_Navigation and Mapping',
       'Industry_Group_Agriculture and Farming',
       'Industry_Group_Natural Resources',
       'Industry_Group_Government and Military', 'Industry_Group_Apps',
       'Industry_Group_Artificial Intelligence',
       'Industry_Group_Science and Engineering']
    
    def fit(self,X,y=None):
        return self
    def transform(self,X,y=None):
        return X.drop(self.weak_features, axis=1)

In [5]:
preproc_numerical= make_pipeline(
    SimpleImputer(),
    RobustScaler()
)

preproc_categorical = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore",sparse=False)
)

preproc = make_column_transformer(
    (preproc_numerical, make_column_selector(dtype_include=["int64", "float64"])),
    (preproc_categorical, make_column_selector(dtype_include=["object"])), remainder="drop")


In [6]:
pipe_baseline = make_pipeline(preproc, RandomForestClassifier(random_state=42))

In [7]:
preproc.fit_transform(X_train)

/Users/deepatamraparani/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[13.84375   ,  7.        ,  3.62323944, ...,  0.        ,
         0.        ,  0.        ],
       [-0.1911725 ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.13592313,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.20416667,  1.        ,  0.10739437, ...,  0.        ,
         0.        ,  0.        ],
       [-0.10416667,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.18007927,  1.        ,  0.25      , ...,  0.        ,
         0.        ,  0.        ]])

In [8]:
new_col_names1 = preproc.named_transformers_['pipeline-1'][0].get_feature_names_out()

new_col_names2 = preproc.named_transformers_['pipeline-2'][1].get_feature_names_out(['country_code','Industry_Group'])

In [9]:
X_train_transformed = pd.DataFrame(preproc.fit_transform(X_train),columns=list(new_col_names1)+ list(new_col_names2))

/Users/deepatamraparani/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
droppertranf = columnDropperTransformer()
X_train_transformed = droppertranf.fit_transform(X_train_transformed)

In [11]:
X_train_transformed.to_csv('clean_data_encoded.csv')

In [12]:
#pipe_baseline.fit(X_train_transformed)

TypeError: Pipeline.fit() missing 1 required positional argument: 'X'